In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# NPBのページURL
url = "https://npb.jp/bis/teams/"

# HTMLを取得
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 文字化け防止
soup = BeautifulSoup(response.text, "html.parser")

# データを格納するリスト
teams_data = []

# `<h4>` タグ（リーグ名）を取得
for h4_tag in soup.find_all("h4"):
    league_name = h4_tag.text.strip()  # リーグ名（例: CENTRAL LEAGUE）

    # `<h4>` タグの次の `<ul>` 内のチーム情報を取得
    ul_tag = h4_tag.find_next("ul")
    if ul_tag:
        for a_tag in ul_tag.find_all("a", href=True):
            team_name = a_tag.text.strip()  # チーム名
            team_url = a_tag["href"]  # チームページURL（相対URL）

            # ロゴ画像 (`.png` を含む `<img>` タグを探す)
            img_tag = a_tag.find("img", src=True)
            logo_url = img_tag["src"] if img_tag and ".png" in img_tag["src"] else None

            # 相対URLを絶対URLに変換
            if not team_url.startswith("http"):
                team_url = f"https://npb.jp/bis/teams/{team_url}"
            if logo_url and not logo_url.startswith("http"):
                logo_url = f"https:{logo_url}"

            # データをリストに追加
            teams_data.append([league_name, team_name, logo_url, team_url])

# pandasでデータフレーム作成
df_team = pd.DataFrame(teams_data, columns=["リーグ名", "チーム名", "ロゴURL", "チームページURL"])

df_team.head(20)

In [ ]:
data = []

for team_name, url in zip(df["チーム名"], df["チームページURL"]):
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # エラーチェック
    
    # 自動エンコーディング適用
    response.encoding = response.apparent_encoding

    # HTMLを解析
    soup = BeautifulSoup(response.text, "html.parser")

    # 選手一覧の取得 (tr class="rosterPlayer")
    players = soup.find_all("tr", class_="rosterPlayer")

    # 各選手の情報を取得
    for player in players:
        columns = player.find_all("td")  # 各列（tdタグ）
        if len(columns) >= 5:  # 必要な情報が入っているか確認
            number = columns[0].text.strip()  # 背番号
            name = columns[1].text.strip()    # 選手名
            height = columns[3].text.strip()  # 身長
            weight = columns[4].text.strip()  # 体重
            data.append([team_name, number, name, height, weight])

df_players = pd.DataFrame(data, columns=["team", "number", "name", "height", "weight"])
df_players.head(10)

In [ ]:
# JSONに変換（辞書形式）
json_data = df_players.to_json(orient="records", force_ascii=False)

# ファイル保存（iOSアプリに渡す）
with open("players.json", "w", encoding="utf-8") as f:
    f.write(json_data)